In [ ]:
#imports
from bs4 import BeautifulSoup
import requests
import pandas as pd

#set ups
#pandas originally cut the data frames, this is used to desplay the full df
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)


In [ ]:
url = 'https://www.hellenicparliament.gr/Vouli-ton-Ellinon/To-Politevma/Ekloges/Eklogika-apotelesmata-New/'
response = requests.get(url)
response

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

In [ ]:
soup.find_all('p')

In [ ]:
urls=[]

for p in soup.find_all('p'):
    for link in p.find_all('a',{'target':'_blank'}):#το target εννοεί το attribute που εχει το href, dhladh to blank
        
        if 'Σύνθεση' in link.text:
#             print(link.text)
            url = link.get('href')

            print(url)
            urls.append(url)
        
        #το href είναι για να πάρω το καθαρό link
        print(link.get('href'))   

In [ ]:
more_urls = []

for p in soup.find_all('p'):
    if 'Σύνθεση' in p.text:
        url = p.find('a',href=True).get('href')
        print(url)

        more_urls.append(url)

In [ ]:
len(more_urls)

In [ ]:
#store the urls you need in a temporary df
#turn part of your more_urls list into a df

temp_df = pd.DataFrame(more_urls[1:])

#rename your first column
temp_df = temp_df.rename(columns={temp_df.columns[0]:'url'})

#check out your df
temp_df.head()

In [ ]:
base_url = 'http://www.hellenicparliament.gr/'

In [ ]:
#create a new column named final_url
temp_df['final_url'] = ''

#iterrate over the rows in your df
for i, item in temp_df.iterrows():
    #if 'hellenicparliament.gr' is already part of your url
    if 'hellenicparliament.gr/' in item['url']: 
        #then you are all set, so your final_url will be equal to the one you already have
        temp_df.at[i, 'final_url'] = item['url']
    #otherwise
    else:
        #we'll construct your final_url by adding your current url to the base_url
        temp_df.at[i, 'final_url'] = base_url+item['url']

In [ ]:
numbers = list(range(1,31))

In [ ]:
len(numbers)

In [ ]:
for n in numbers:
    print(n)

In [ ]:
#to navigate in this table, you normally click on different pages numbers in the pagination list at the bottom of the table
#each time you click on a different table page number, your url changes! But it changes in a predictable way 
#so, we can construct all the different urls you need to visit in order to navigate the whole list of the MPs

#let's do this for the first 9 rows of your temp_df
#meaning let's do this for the last 8 parliamentary periods which go back to 1996

webpages = []

for url in temp_df.final_url[0:9]:
    if 'SInthesi-Proigoumenon-Periodon' in url:
        for n in numbers[0:15]:
            webpage = url+'&pageNo='+str(n)
            webpages.append(webpage)

    else:
        for n in numbers[0:30]:
            webpage = url+'?pageNo='+str(n)

            webpages.append(webpage)
        
webpages

In [ ]:
#turn this full list of the webpages you wish to visit into a df
df1 = pd.DataFrame(webpages)
df1 = df1.rename(columns={df1.columns[0]:'url'})
df1

In [ ]:
import time

In [ ]:
#let's scrape the names of the MPs elected in the last 10 election rounds
#do it all at once

#create an empty list
names = []

#loop through your urls
#for each one of those
for url in df1.url:
    #make a request and visit it
    response = requests.get(url)
    #make python your script sleep for 3 seconds before making the next request
    time.sleep(3)
    #parse each one of the pages
    soup_doc = BeautifulSoup(response.text, 'html.parser')
    
#     try:
    #grab all the rows found in the table on each one of the webpages you visit
    rows = soup_doc.find('table').find('tbody').find_all('tr')
#     except:
#         pass
    
    #grap all the cell values for each one of the rows you got but for the last row in each table (with is the pagination bar)
    for row in rows[:-1]:
        cells  = row.find_all('td')

        if 'SInthesi-Proigoumenon-Periodon' in url:
            for cell in cells[0::3]:
                print(cell.text.strip())
                names.append(cell.text.strip())
                
        else:
#           loop through your cells that hold MP's names
            for cell in cells[1::3]:
#               print the names as you scrape them
                print(cell.text)
#               append your names list with the cell values (names) you are getting
                names.append(cell.text.strip())

In [ ]:
9*300

In [ ]:
df.names.value_counts()